<a href="https://colab.research.google.com/github/hillsonghimire/pyGroundWaterLoss_GRACE-FO/blob/main/gee-injestion-util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
GRACE Util version 1.0 (Feb 2024)
GRACE/GRACE-FO CSR data conversion and Earth Engine data injestion pipeline
2024: Hillson Ghimire

Revisions:

'''

### Install and import necessary libraries. Initialize Earth Engine

In [2]:
!pip install "xarray[complete]" &> /dev/null
# !pip install rasterio &> /dev/null
# !pip install netcdf4 &> /dev/null

!pip install pycrs &> /dev/null
# !pip install geemap &> /dev/null
!pip install cftime &> /dev/null
!pip install rioxarray &> /dev/null

import ee
import os
import json
import cftime
import geemap
import datetime
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio
from datetime import datetime
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

ee.Authenticate()
ee.Initialize(project= 'gee-ait')

Mounted at /content/drive


### Module to download GRACE/GRACE-FO(in netcdf) format and export as geotiff

In [3]:
!mkdir GRACE
!mkdir GRACE_CSV

In [4]:
## Uncomment to download data
# ! wget http://download.csr.utexas.edu/outgoing/grace/RL0602_mascons/CSR_GRACE_GRACE-FO_RL0602_Mascons_all-corrections.nc

#### Export CSV files with time properties of whole GRACE/GRACE-FO collections

In [5]:
path = '/content/drive/MyDrive/Colab Notebooks/FAO Analysis/data/CSR_GRACE_GRACE-FO_RL0602_Mascons_all-corrections.nc'
ds = xr.open_dataset(path)

# Get the time and time_blunds variables and units
time = ds['time']
# print(time)
# Define a start date as July 2, 2014
timeDelta = datetime(2002, 1, 1, 0, 0, 0) - datetime(1970, 1, 1, 0, 0, 0)
first_day_ms = timeDelta.total_seconds()*1000

time_units = time.attrs['Units']
time_bound = ds['time_bounds']
time_bound_ar = time_bound.values

# the data is in format [[start, end], [start, end]....]
# first extract list as [start, start ...] and [end, end ...]
# and then convert the list back to xarray.DataArray type
time_bound_pre = time_bound_ar[:, :1]
time_bound_pre = xr.DataArray([float(sublist[0]) for sublist in time_bound_pre])

time_bound_post = time_bound_ar[:, 1:]
time_bound_post = xr.DataArray([float(sublist[0]) for sublist in time_bound_post])

# Decode time values into human-readable format
def timeToMs(t, time_units = time_units):
  datetime_ms = [int(first_day_ms+days*86400000)  for days in t.values]
  return datetime_ms

# Time conversion step
time                 = timeToMs(time)
time_bound_pre       = timeToMs(time_bound_pre)
time_bound_post      = [(excludeLastDay-86400000) for excludeLastDay in timeToMs(time_bound_post)]
time_bound_pre_date  = [(datetime.utcfromtimestamp(x / 1000)).strftime('%Y-%m-%d').replace('-', '') for x in time_bound_pre]
time_bound_post_date = [(datetime.utcfromtimestamp(x / 1000)).strftime('%Y-%m-%d').replace('-', '') for x in time_bound_post]
# get the final list of time, time_bound_pre, and time_bound_post
time_time_bounds = list(zip(time, time_bound_pre, time_bound_pre_date, time_bound_post, time_bound_post_date))
time_time_bounds = pd.DataFrame.from_records(time_time_bounds, columns =['time', 'time_beginning_ms', 'time_beginning_YMD', 'time_end_ms', 'time_end_YMD'])
time_time_bounds["id"] = time_time_bounds.index + 1

# Export id to csv
time_time_bounds.to_csv('/content/GRACE_CSV/id_info.csv', index=False)

#### Export masked geotiff, the masking is done utilizing the landmask layer provided by CSR

In [6]:
# Open the GRACE_GRACE-FO_Mascon dataset
path = '/content/drive/MyDrive/Colab Notebooks/FAO Analysis/data/CSR_GRACE_GRACE-FO_RL0602_Mascons_all-corrections.nc'
ds = xr.open_dataset(path)

# Extract latitude, longitude, and time dimensions
lat = ds['lat'].values
lon = ds['lon'].values
time = ds['time'].values
# Get dimensions
height, width = len(lat), len(lon)

def _coordAssign(xrData):
  # convert longitude from (0-360) to (-180 to 180) (if required)
  xrData['lon'] = (xrData['lon'] + 180) % 360 - 180
  xrData = xrData.sortby(xrData.lon)
  xrData = xrData.rio.set_spatial_dims('lon', 'lat')
  xrData.rio.set_crs("epsg:4326")                                 ## print(data.rio.crs) #if CRS isn't discovered or None, add CRS using data.rio.set_crs(<YOUR EPSG>)
  return xrData

# Land mask layer
mask = True
nodata_value = np.nan
if mask==True:
  path_mask = '/content/drive/MyDrive/Colab Notebooks/FAO Analysis/data/CSR_GRACE_GRACE-FO_RL06_Mascons_v02_LandMask.nc'
  ds_mask = xr.open_dataset(path_mask)
  ds_mask1 = xr.where(ds_mask == 0, nodata_value, ds_mask)
  ds_mask2 = _coordAssign(ds_mask1)                                ## Assign Coordinates
  # <DATA-NAME>.rio.to_raster(f"/content/LANDMASK/landmask.tif")   ## To export mask as geotiff

# To mask out ocean
def maskFunc(toClip, ds_mask = ds_mask):
  return xr.where(ds_mask==1, toClip, float('nan'))

# Iterate over each time step
for i, t in enumerate(time):
  # Extract data for the current time step
  data = ds['lwe_thickness'][i]

  if mask == True:
    data = maskFunc(data)
  data = _coordAssign(data)                                        ## Assign Coordinates

  # Construct the output file name
  pre_date = time_time_bounds['time_beginning_YMD'][i]
  post_date = time_time_bounds['time_end_YMD'][i]
  output_filename = f"/content/GRACE/{pre_date}_{post_date}.tif"   ## Can adjust name format
  data.rio.to_raster(output_filename)

# Zip the data folder
!zip -r GRACE.zip /content/GRACE/*.tif >> /dev/null

### Generate manifest.json file to injest the data to earth engine


In [ ]:
!mkdir json

In [ ]:
# setup paths and environment
output_path = '/content/json/'
cloud_path = 'gs://gee-ingest-fao/GRACE'
gee_asset_path = 'projects/gee-ait/assets/GRACE/MASS_GRIDS/LAND'
manifest_dir = output_path

# list all the image data, the manifest will be based on image properties
lst = os.listdir('/content/GRACE/')

#----------------------------------------------------------------------

# manifest generator
for i, image in enumerate(lst):
  tileset_id = str(time_time_bounds['time_beginning_YMD'][i])+'_'+str(time_time_bounds['time_end_YMD'][i])
  manifest_name = '{}/{}'.format(gee_asset_path, tileset_id)

  manifest_tilesets = []
  manifest_bands = []
  # print(manifest_name)

  _tileset = {
      'id': tileset_id,
      'sources': [
          {
              'uris': [
                  '{}/{}'.format(cloud_path, image)
              ]
          }
      ]
  }

  _band = {
    'id': "lwe_thickness_csr",
    'tileset_id': tileset_id
  }

  manifest_tilesets.append(_tileset)
  manifest_bands.append(_band)

  ## properties
  manifest_properties = {}
  manifest_properties['CSR_START_TIME'] = int(time_time_bounds['time_beginning_ms'][i])
  manifest_properties['CSR_end_TIME'] = int(time_time_bounds['time_end_ms'][i])
  # start_index = 3
  # for _property in properties:
  #     manifest_properties[_property] = image[start_index]
  #     start_index += 1

  # start time
  manifest_start_time = {
      'seconds': int(time_time_bounds['time_beginning_ms'][i]/1000)
  }

  # end time
  manifest_end_time = {
      'seconds': int(time_time_bounds['time_end_ms'][i]/1000)
  }

  final_manifest = {
    'name': manifest_name,
    'tilesets': manifest_tilesets,
    'bands': manifest_bands,
    'start_time': manifest_start_time,
    'end_time': manifest_end_time,
    'properties': manifest_properties
  }
  # print(final_manifest)
  with open('{}{}.json'.format(manifest_dir, tileset_id), 'w') as manifest_file:
    json_string = json.dumps(final_manifest, ensure_ascii=False, indent=4)
    manifest_file.write(json_string)

!zip -r jsonGrace.zip /content/json/*.json >> /dev/null